In [12]:
# load module
from hdf5storage import loadmat, savemat
import numpy as np 


from scipy import signal 
from scipy.fftpack import fft
from scipy import stats # for zscoring


In [2]:
# load data
def loaddata(filename):
    data = loadmat(filename)
    bpchan = int(data['bpchan'][0][0])
    channels = data['channels'][0]
    conditionNames = data['conditionNames'][0]
    conditions = data['conditions'][0]
    eeg = dict()
    eeg[0] = data['dataL'][0]
    eeg[1] = data['dataR'][0]
    intervals = data['intervals'][0]
    labels = data['labels'][0]
    samples = data['samples'][0]
    session = int(data['session'][0])
    sessionTypes = data['sessionTypes'][0]
    sr = int(data['sr'][0])
    #Labels and condition names doesnt come over cleanly from matlab 
    chan_labels = list()
    condition_Names = list()
    #pythonify condition index
    condition_index = list()
    for j in range(len(labels)):
        chan_labels.append(labels[j][0])
    for j in range(len(conditionNames)):
        condition_Names.append(conditionNames[j][0])
    for j in range(12):
        condition_index.append(int(conditions[j]-1))
    return eeg, intervals, samples, condition_index, session, sr, bpchan, condition_Names, channels, chan_labels, sessionTypes   


In [3]:
def avref(datax):
	ref = np.mean(datax,axis = 1)
	refmat = np.tile(ref,(np.shape(datax)[1],1))
	datax = datax - np.transpose(refmat)
	return datax

In [4]:
def segmenteegdata(eegdata,epochlength,samplingrate):
    epoch = int(epochlength*samplingrate)
    nsamp = np.shape(eegdata)[0]
    nchan = np.shape(eegdata)[1]
    nepoch = int(nsamp/epoch)
    nsamp = nepoch*epoch
    segdata = np.reshape(eegdata[0:nsamp,0:nchan],(nepoch,epoch,nchan))
    return segdata


In [6]:
# organize file in time sequence
filedates=[20220713,20220721,20220804,20220808,20220810,20220811,20220815,20220816,20221003,2022100401,2022100402,20221005]
numSes=len(filedates)

In [7]:
# directory
filedir="/home/zhibinz2/zhibin/analysis/" # your own data directory
pathname = 'Cleaned_data/' # our common folder name
dir=filedir+pathname

In [8]:
# parameters 
epoch = 1 #units: seconds
maxf = 30 #units: Hz
maxbin = maxf*epoch #convert maxf into number of bins to save


In [9]:
# get fceof in sequence of 12 ses x 12 trl x 2 suj

abs_mean_all=np.zeros((12,12,2,30))
abs_std_all=np.zeros((12,12,2,30))
abs_zscore_all=np.zeros([12,12,2]).tolist()

for ses in range(12):
	filename='clean_'+str(filedates[ses])+'.mat'
	[eeg, intervals, samples, condition_index, session, sr, bpchan, 
   		condition_Names, channels, chan_labels, sessionTypes] \
			= loaddata(dir+filename)
	# clear output arrays 
	power_mat = np.zeros((2,12,maxbin,32))

	abs_mean_mat = np.zeros((12,12,2,maxbin,32))
	abs_abs_mat = np.zeros((12,12,2,maxbin,32))

	amp_corr_mat = np.zeros((2,12,maxbin,32,32))
	c_spect_mat = np.zeros((2,12,maxbin,32,32),dtype='complex')
	coh_mat = np.zeros((2,12,maxbin,32,32))
	pcorr_mat = np.zeros((2,12,maxbin,32,32))
	pcorr_cov_mat = np.zeros((2,12,maxbin,32,32))
	pcorr_alphas = np.zeros((2,12,maxbin))
	for trl in range(12):
		#align the length of the two files. 
		nsamp1 = np.shape(eeg[0][trl])[0]
		nsamp2 = np.shape(eeg[1][trl])[0]
		nsamp = np.min((nsamp1,nsamp2))
		for subj in range(2): # for each subject
			trialdata = avref(eeg[subj][trl][0:nsamp,0:32]) 
			trialdata = segmenteegdata(trialdata,epoch,sr)
			trialdata = signal.detrend(trialdata,axis =1)
			dataf = fft(trialdata,axis = 1)
			fcoef = dataf[:,1:maxbin+1,:]/(epoch*sr)/epoch

			abs_mean =np. mean(np.abs(fcoef),axis =0)
			abs_std =np. mean(np.abs(fcoef),axis =0)
			abs_zscore = stats.zscore(np.abs(fcoef),axis=0) 

			abs_mean_mat[ses,trl,subj,:,:] = abs_mean
			abs_abs_mat[ses,trl,subj,:,:] = abs_std
			abs_zscore_all[ses][trl][subj] = abs_zscore



In [13]:
outdict=dict()
outdict['abs_zscore_all']=abs_zscore_all
savemat('abs_zscore_all',outdict,store_python_metadata=True)